In [9]:
!pip install pdfplumber pandas pymupdf



In [8]:
import pdfplumber, os

pdf_folder = "pdfs"  # folder with 10 PDFs
all_texts = []

for file in os.listdir(pdf_folder):
    if file.endswith(".pdf"):
        with pdfplumber.open(os.path.join(pdf_folder, file)) as pdf:
            text = ""
            for page in pdf.pages:
                text += page.extract_text() or ""
            all_texts.append({"file": file, "text": text[:2000]})  # preview first 2000 chars

# Show preview of extracted text
all_texts[:2]


[{'file': '10.41467_2025_Article_59047.pdf',
  'text': 'Article https://doi.org/10.1038/s41467-025-59047-z\nMechanosensing antagonizes ethylene\nsignaling to promote root gravitropism\nin rice\nReceived:25October2024 Han-QingWang 1,Xing-YuZhao1,ZhongTang1,Xin-YuanHuang 1,\nPengWang 1,WenhuaZhang 2,YunhuiZhang3,ShengLuan 4&\nAccepted:7April2025\nFang-JieZhao 1\nCheckforupdates Rootgravitropismreliesongravityperceptionbytherootcapandrequires\ntightlyregulatedphytohormonesignaling.Here,weisolatearicemutantthat\ndisplaysrootcoilinginhydroponicsbutnormalgravitropicgrowthinsoil.We\nidentifyCOILINGROOTINWATER1(CRW1)encodinganETHYLENE-\nINSENSITIVE3(EIN3)-BINDINGF-BOXPROTEIN(OsEBF1)asthecausativegene\nforthemutantphenotype.WeshowthattheOsCRW1-EIN3LIKE1and2\n(OsEIL1/2)-ETHYLENERESPONSEFACTOR82(OsERF82)modulecontrolsthe\nproductionofreactiveoxygenspeciesintheroottip,subsequentlyimpacting\nrootcapstability,polarlocalizationofPIN-FORMED2(OsPIN2),symmetric\ndistributionofauxin,andultimatelygravitro

In [10]:
import re
import pandas as pd

dataset = []

for idx, doc in enumerate(all_texts, start=1):
    text = doc["text"]

    # Title = first line
    title = text.split("\n")[0][:200]

    # Try to find abstract
    abstract_match = re.search(r"Abstract(.*?)(Introduction|Methods|Materials)", text, re.S | re.I)
    abstract = abstract_match.group(1).strip() if abstract_match else ""

    # Try to find conclusion
    conclusion_match = re.search(r"(Conclusion|Summary)(.*?)(References|Acknowledgment|$)", text, re.S | re.I)
    conclusion = conclusion_match.group(2).strip() if conclusion_match else ""

    dataset.append({
        "id": idx,
        "title": title,
        "species": "",          # will fill later manually
        "environment": "",      # will fill later manually
        "effect": "",           # will fill later manually
        "abstract": abstract,
        "conclusion": conclusion,
        "source_file": doc["file"]
    })

df = pd.DataFrame(dataset)
df.to_csv("bio_experiments.csv", index=False)
df.head()


,id,title,species,environment,effect,abstract,conclusion,source_file
0,1,Article https://doi.org/10.1038/s41467-025-590...,,,,,,10.41467_2025_Article_59047.pdf
1,2,www.nature.com/scientificreports,,,,,", this study underscores the potential of Wolf...",8.41598_2023_Article_49680.pdf
2,3,Cell Structure and Function Cell Structure and...,,,,,,4.csf_49_24035.pdf
3,4,cancers,,,,,,1.cancers-12-00381.pdf
4,5,Experimental Brain Research (2025) 243:127,,,,Under conditions of weightlessness human posit...,,9.221_2025_Article_7090.pdf


In [11]:
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.7 MB/s eta 0:00:00


In [12]:
import os
import pandas as pd
from PyPDF2 import PdfReader
import re
import spacy

# Load small English NLP model
nlp = spacy.load("en_core_web_sm")  # Make sure to install: pip install spacy && python -m spacy download en_core_web_sm

# Create or load dataset
dataset_file = "final_dataset.csv"
if os.path.exists(dataset_file):
    df = pd.read_csv(dataset_file)
else:
    df = pd.DataFrame(columns=[
        "id", "title", "species", "environment", "effect",
        "abstract", "conclusion", "source_file", "full_text"
    ])

# Example keywords lists for detection
species_list = ["E. coli", "S. aureus", "B. subtilis"]
environment_list = ["soil", "water", "air", "marine"]
effect_list = ["toxic", "beneficial", "neutral"]

# Function to extract full text
def extract_pdf_text(pdf_path):
    text = ""
    try:
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
    return text

# Function to extract a section based on heading
def extract_section(text, section_names):
    for section in section_names:
        pattern = re.compile(rf"{section}\s*[:\-]?\s*(.*?)\n(?:\n|[A-Z][a-z])", re.IGNORECASE | re.DOTALL)
        match = pattern.search(text)
        if match:
            return match.group(1).strip()
    return None

# Folder containing PDFs
pdf_folder = "pdfs"  # replace with your folder path

# Process PDFs
for idx, file in enumerate(os.listdir(pdf_folder), start=1):
    if file.lower().endswith(".pdf"):
        path = os.path.join(pdf_folder, file)
        full_text = extract_pdf_text(path)
        full_text_clean = full_text.replace("\n", " ").strip()

        # NLP processing for keywords
        doc = nlp(full_text_clean)
        species_found = [s for s in species_list if s.lower() in full_text_clean.lower()]
        environment_found = [e for e in environment_list if e.lower() in full_text_clean.lower()]
        effect_found = [ef for ef in effect_list if ef.lower() in full_text_clean.lower()]

        # Extract abstract and conclusion
        abstract = extract_section(full_text, ["Abstract"])
        conclusion = extract_section(full_text, ["Conclusion", "Conclusions"])

        # Extract title (try first line or from filename)
        title = file.replace(".pdf", "")
        first_line = full_text.split("\n")[0].strip()
        if len(first_line) > 5:  # if first line looks like a title
            title = first_line

        # Append row
        df = pd.concat([df, pd.DataFrame([{
            "id": idx,
            "title": title,
            "species": ", ".join(species_found) if species_found else None,
            "environment": ", ".join(environment_found) if environment_found else None,
            "effect": ", ".join(effect_found) if effect_found else None,
            "abstract": abstract,
            "conclusion": conclusion,
            "source_file": file,
            "full_text": full_text_clean
        }])], ignore_index=True)

# Save final dataset
df.to_csv(dataset_file, index=False)
print(f"Dataset saved! Total PDFs processed: {len(df)}")


Dataset saved! Total PDFs processed: 10


In [13]:
print(df.head())


  id                                              title  species  \
0  1  Article https://doi.org/10.1038/s41467-025-590...  E. coli   
1  2                         8.41598_2023_Article_49680     None   
2  3                        Cell Structure and Function     None   
3  4                                            cancers     None   
4  5  RESEARCH ARTICLEExperimental Brain Research (2...     None   

        environment          effect  \
0  soil, water, air         neutral   
1        water, air         neutral   
2        water, air  toxic, neutral   
3               air           toxic   
4               air         neutral   

                                            abstract  \
0                                               None   
1                                               None   
2  s Anhydrobiosis, a phenomenon in which organis...   
3  Background: Ionizing radiation from galactic c...   
4  Under conditions of weightlessness human posit...   

                   

In [14]:
import spacy
import pandas as pd

# Load NLP model
nlp = spacy.load("en_core_web_sm")

# Load your dataset
df = pd.read_csv("final_dataset.csv")

# Example: expand species list
species_list = ["E. coli", "S. aureus", "B. subtilis", "R. varieornatus"]

# Fill missing abstracts
for i, row in df.iterrows():
    if pd.isna(row['abstract']) or row['abstract'] == "None":
        text = row['full_text']
        # Look for sentences containing the word 'abstract' or beginning part of the text
        match = re.search(r"(?:Abstract[:\-\s]*)?(.{50,500})", text, re.IGNORECASE | re.DOTALL)
        if match:
            df.at[i, 'abstract'] = match.group(1).strip()

# Fill missing species
for i, row in df.iterrows():
    if pd.isna(row['species']) or row['species'] == "None":
        text = row['full_text'].lower()
        found = [s for s in species_list if s.lower() in text]
        if found:
            df.at[i, 'species'] = ", ".join(found)

# Save updated dataset
df.to_csv("final_dataset_complete.csv", index=False)
print("Dataset updated and saved as final_dataset_complete.csv")


Dataset updated and saved as final_dataset_complete.csv


In [15]:
print(df.head())


   id                                              title          species  \
0   1  Article https://doi.org/10.1038/s41467-025-590...          E. coli   
1   2                         8.41598_2023_Article_49680              NaN   
2   3                        Cell Structure and Function  R. varieornatus   
3   4                                            cancers              NaN   
4   5  RESEARCH ARTICLEExperimental Brain Research (2...              NaN   

        environment          effect  \
0  soil, water, air         neutral   
1        water, air         neutral   
2        water, air  toxic, neutral   
3               air           toxic   
4               air         neutral   

                                            abstract  \
0  Article https://doi.org/10.1038/s41467-025-590...   
1  1 Vol.:(0123456789) Scientific Reports  |     ...   
2  s Anhydrobiosis, a phenomenon in which organis...   
3  Background: Ionizing radiation from galactic c...   
4  Under conditions of

In [16]:
from google.colab import files
files.download("final_dataset_complete.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
df_preview = df.copy()
df_preview['full_text'] = df_preview['full_text'].str[:200]  # first 200 chars
df_preview.to_csv("dataset_preview.csv", index=False)


In [18]:
from google.colab import files
files.download("dataset_preview.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
!pip install transformers torch pandas --quiet


In [21]:
import pandas as pd

# Load your preprocessed dataset
df = pd.read_csv("final_dataset_complete.csv")
df.head()


,id,title,species,environment,effect,abstract,conclusion,source_file,full_text
0,1,Article https://doi.org/10.1038/s41467-025-590...,E. coli,"soil, water, air",neutral,Article https://doi.org/10.1038/s41467-025-590...,NaN,10.41467_2025_Article_59047.pdf,Article https://doi.org/10.1038/s41467-025-590...
1,2,8.41598_2023_Article_49680,NaN,"water, air",neutral,1 Vol.:(0123456789) Scientific Reports | ...,", this study underscores the potential of Wolf...",8.41598_2023_Article_49680.pdf,1 Vol.:(0123456789) Scientific Reports | ...
2,3,Cell Structure and Function,R. varieornatus,"water, air","toxic, neutral","s Anhydrobiosis, a phenomenon in which organis...","In this study, we expressed R. varieornatus C...",4.csf_49_24035.pdf,Cell Structure and Function Cell Structure and...
3,4,cancers,NaN,air,toxic,Background: Ionizing radiation from galactic c...,s: We provided a,1.cancers-12-00381.pdf,cancers Article NASA GeneLab Platform Utilized...
4,5,RESEARCH ARTICLEExperimental Brain Research (2...,NaN,air,neutral,Under conditions of weightlessness human posit...,strengthened our view that spindles played a key,9.221_2025_Article_7090.pdf,RESEARCH ARTICLEExperimental Brain Research (2...


In [22]:
import re

def clean_text(text):
    if not isinstance(text, str):
        return ""
    # Remove extra spaces, line breaks, and weird characters
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


In [23]:
from transformers import pipeline
import pandas as pd
import re

df = pd.read_csv("final_dataset_complete.csv")

summarizer = pipeline("summarization", model="google/pegasus-xsum")

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def chunk_text(text, max_tokens=200):
    words = text.split()
    for i in range(0, len(words), max_tokens):
        yield " ".join(words[i:i+max_tokens])

def summarize_full_text(text):
    text = clean_text(text)
    if len(text.split()) < 30:
        return "Text too short"
    try:
        chunks = list(chunk_text(text))
        summaries = []
        for chunk in chunks:
            result = summarizer(
                chunk,
                max_length=80,
                min_length=30,
                do_sample=False
            )
            summaries.append(result[0]['summary_text'])
        return " ".join(summaries)
    except Exception as e:
        return f"Error: {str(e)}"

df['ai_summary'] = df['full_text'].fillna("").apply(summarize_full_text)
df.to_csv("dataset_with_fulltext_summaries.csv", index=False)
print("✅ Summaries created using Pegasus model")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Token indices sequence length is longer than the specified maximum sequence length for this model (625 > 512). Running this sequence through the model will result in indexing errors


✅ Summaries created using Pegasus model


In [24]:
import os

files_to_remove = [
    "dataset_with_summaries.csv",
    "dataset_with_fulltext_summaries.csv",
    "dataset_with_detailed_summaries.csv"
]

for f in files_to_remove:
    if os.path.exists(f):
        os.remove(f)
        print(f"Deleted {f}")



Deleted dataset_with_fulltext_summaries.csv


In [25]:
!pip install sumy
!pip install nltk


In [26]:
import nltk
nltk.download('punkt')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [27]:
import nltk

# Standard punkt tokenizer
nltk.download('punkt')

# Some versions require punkt_tab for Sumy
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [28]:
from nltk.data import find

# Check if punkt is installed
find('tokenizers/punkt')
find('tokenizers/punkt_tab')  # optional


FileSystemPathPointer('/root/nltk_data/tokenizers/punkt_tab')

In [29]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

parser = PlaintextParser.from_string("Your text here", Tokenizer("english"))


In [30]:
import pandas as pd

df = pd.read_csv("final_dataset_complete.csv")
df.head()
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
import re

# Clean text function
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Initialize summarizer
summarizer = LexRankSummarizer()

# Summarize full text
def summarize_full_text(text, sentence_count=10):
    text = clean_text(text)
    if len(text.split()) < 30:
        return "Text too short for summarization"
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summary = summarizer(parser.document, sentences_count=sentence_count)
    return " ".join([str(sentence) for sentence in summary])


In [31]:
# Create a new column for summaries
df['ai_summary'] = df['full_text'].apply(lambda x: summarize_full_text(x, sentence_count=10))

# Save the new CSV
df.to_csv("dataset_with_extractive_summaries.csv", index=False)
print("✅ Extractive summaries created and saved!")


✅ Extractive summaries created and saved!


In [32]:
pd.set_option('max_colwidth', None)  # show full summary text
df[['title', 'ai_summary']].head()


title  \
0          Article https://doi.org/10.1038/s41467-025-59047-z   
1                                  8.41598_2023_Article_49680   
2                                 Cell Structure and Function   
3                                                     cancers   
4  RESEARCH ARTICLEExperimental Brain Research (2025) 243:127   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [33]:
df.to_json("dataset.json", orient="records")


In [34]:
import pandas as pd

df = pd.read_csv("dataset_with_extractive_summaries.csv")

# Show first 5 rows
print(df.head())

# Show full summaries without truncation
pd.set_option('max_colwidth', None)
print(df[['title', 'ai_summary']].head())


   id                                                       title  \
0   1          Article https://doi.org/10.1038/s41467-025-59047-z   
1   2                                  8.41598_2023_Article_49680   
2   3                                 Cell Structure and Function   
3   4                                                     cancers   
4   5  RESEARCH ARTICLEExperimental Brain Research (2025) 243:127   

           species       environment          effect  \
0          E. coli  soil, water, air         neutral   
1              NaN        water, air         neutral   
2  R. varieornatus        water, air  toxic, neutral   
3              NaN               air           toxic   
4              NaN               air         neutral   

                                                                                                                                                                                                                                                        

In [35]:
!pip install evaluate rouge-score

import evaluate

# Load ROUGE metric
rouge = evaluate.load("rouge")

predictions = ["AI generated summary here"]
references = ["Ground truth abstract or human summary"]

# Compute ROUGE
results = rouge.compute(predictions=predictions, references=references)
print(results)



  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.0 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9ddec0f20629d6f84fd0dce6d64779f4065cd215e5745f130bf45d67cdc5441b
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


{'rouge1': np.float64(0.2), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.2), 'rougeLsum': np.float64(0.2)}


In [38]:
# 📌 Install required libraries (run once in Colab)
!pip install sumy

# ------------------------------
# Imports
# ------------------------------
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

LANGUAGE = "english"

# ------------------------------
# Explore Mode (short, human-like)
# ------------------------------
def generate_explore_summary(text, sentence_count=3):
    parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))
    summarizer = LsaSummarizer(Stemmer(LANGUAGE))
    summarizer.stop_words = get_stop_words(LANGUAGE)

    summary = summarizer(parser.document, sentence_count)
    return " ".join([str(sentence) for sentence in summary])

# ------------------------------
# Research Mode (longer, technical)
# ------------------------------
def generate_research_summary(text, sentence_count=8):
    parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))
    summarizer = LsaSummarizer(Stemmer(LANGUAGE))
    summarizer.stop_words = get_stop_words(LANGUAGE)

    summary = summarizer(parser.document, sentence_count)
    return " ".join([str(sentence) for sentence in summary])

# ------------------------------
# Example Usage
# ------------------------------
sample_text = """
Artificial Intelligence (AI) is transforming healthcare through advanced diagnostic tools,
predictive analytics, and personalized treatment recommendations. Recent research explores
the application of deep learning in radiology, cardiology, and oncology. However, challenges
remain in data privacy, interpretability, and clinical validation. Future studies must address
ethical concerns while improving accuracy and scalability across diverse populations.
"""

print("🔹 Explore Mode Summary:")
print(generate_explore_summary(sample_text))

print("\n🔹 Research Mode Summary:")
print(generate_research_summary(sample_text))


🔹 Explore Mode Summary:
Artificial Intelligence (AI) is transforming healthcare through advanced diagnostic tools, predictive analytics, and personalized treatment recommendations. Recent research explores the application of deep learning in radiology, cardiology, and oncology. Future studies must address ethical concerns while improving accuracy and scalability across diverse populations.

🔹 Research Mode Summary:
Artificial Intelligence (AI) is transforming healthcare through advanced diagnostic tools, predictive analytics, and personalized treatment recommendations. Recent research explores the application of deep learning in radiology, cardiology, and oncology. However, challenges remain in data privacy, interpretability, and clinical validation. Future studies must address ethical concerns while improving accuracy and scalability across diverse populations.


In [39]:
sample_text = """
Artificial Intelligence (AI) is transforming healthcare through advanced diagnostic tools,
predictive analytics, and personalized treatment recommendations. Recent research explores
the application of deep learning in radiology, cardiology, and oncology. However, challenges
remain in data privacy, interpretability, and clinical validation. Future studies must address
ethical concerns while improving accuracy and scalability across diverse populations.
"""

# Run Explore Mode
explore_summary = generate_explore_summary(sample_text)
print("🔹 Explore Mode Summary:\n", explore_summary)

# Run Research Mode
research_summary = generate_research_summary(sample_text)
print("\n🔹 Research Mode Summary:\n", research_summary)


🔹 Explore Mode Summary:
 Artificial Intelligence (AI) is transforming healthcare through advanced diagnostic tools, predictive analytics, and personalized treatment recommendations. Recent research explores the application of deep learning in radiology, cardiology, and oncology. Future studies must address ethical concerns while improving accuracy and scalability across diverse populations.

🔹 Research Mode Summary:
 Artificial Intelligence (AI) is transforming healthcare through advanced diagnostic tools, predictive analytics, and personalized treatment recommendations. Recent research explores the application of deep learning in radiology, cardiology, and oncology. However, challenges remain in data privacy, interpretability, and clinical validation. Future studies must address ethical concerns while improving accuracy and scalability across diverse populations.


In [40]:
import pandas as pd

# Load your CSV
df = pd.read_csv("dataset_with_extractive_summaries.csv")  # replace with your actual filename

# Pick one paper (first row for example)
paper_text = df.loc[0, "full_text"]  # replace column name if different

explore_summary = generate_explore_summary(paper_text)
print("🔹 Explore Mode Summary:\n", explore_summary)

# Run Research Mode (7–8 sentences)
research_summary = generate_research_summary(paper_text)
print("\n🔹 Research Mode Summary:\n", research_summary)


🔹 Explore Mode Summary:
 Incontrast, enhanced ethylene concentration from ACC addition mayrepress the translation of OsCRW1/OsEBF1 and thus stabilize OsEIL1and OsEIL2 proteins, as has been shown in Arabidopsis 43.W h e ng r o w n in soil, the shorter and curvy roots of OsEIL1 andOsEIL2 overexpressing lines were largely rescued (Supplementary Fig. The Yeast one-hybrid (Y1H) and electrophoretic mobility shift assay(EMSA) demonstrated that OsERF82 speci ﬁcally bound to the binding sites of the DRE/CRT, GCC-box and G-box in the promoter region ofOsRBOHH and three Class III OsPRX genes (Supplementary Fig. The pro- tein and probe were incubated with EMSA reaction solution (preparedaccording to the manufacturer ’s protocol, Beyotime, China) for 20 min at room temperature, separated on a 5% polyacrylamide native gel at4 °C, and transferred to a nylon membrane.

🔹 Research Mode Summary:
 Changes in nutrient supply, solution pH,aeration (dissolved oxygen level), and exposure to light did not alt

In [41]:
import pandas as pd
from tqdm import tqdm

# Load your CSV (make sure it has a column like "full_text")
df = pd.read_csv("dataset_with_extractive_summaries.csv")  # change to your file name

# Add empty columns for summaries
df["explore_summary"] = ""
df["research_summary"] = ""

# Generate summaries for each row
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row["full_text"]  # replace with your column name

    try:
        # Short summary (Explore Mode)
        df.at[i, "explore_summary"] = generate_explore_summary(text)

        # Detailed summary (Research Mode)
        df.at[i, "research_summary"] = generate_research_summary(text)

    except Exception as e:
        print(f"Error at row {i}: {e}")
        df.at[i, "explore_summary"] = "Error generating explore summary"
        df.at[i, "research_summary"] = "Error generating research summary"

# Save to new CSV
df.to_csv("dataset_with_explore_and_research_summaries.csv", index=False)

print("✅ All summaries generated and saved successfully!")


100%|██████████| 10/10 [01:07<00:00,  6.72s/it]

✅ All summaries generated and saved successfully!


In [42]:
# Preview first 3 rows
preview = pd.read_csv("dataset_with_explore_and_research_summaries.csv")
print(preview[["id", "title", "explore_summary", "research_summary"]].head(3))


   id                                               title  \
0   1  Article https://doi.org/10.1038/s41467-025-59047-z   
1   2                          8.41598_2023_Article_49680   
2   3                         Cell Structure and Function   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   